In [37]:
import requests
from importlib import reload
from secretslocal import COOKIE, VIRTUAGYM_API_KEY, VIRTUAGYM_CLUB_SECRET, CLUB_ID
import datetime
import pandas as pd
from data_structures.user import User 
import re
from typing import Any, Optional
import json 


In [38]:
def get_all_club_members() -> dict:
    url = f"https://api.virtuagym.com/api/v1/club/{CLUB_ID}/member?api_key={VIRTUAGYM_API_KEY}&club_secret={VIRTUAGYM_CLUB_SECRET}&sync_from=0"
    ret = requests.get(url)
    ret_str = ret.content.decode('utf-8')
    data = json.loads(ret_str)
    user_data = data.get('result', [])

    users = {}

    for item in user_data:
        user = User(
            item["member_id"],
            item["firstname"],
            item["lastname"],
            item["active"],
            item["is_pro"],
            item["gender"],
            item["email"],
            item["member_since"],
            item["timestamp_edit"],
            item["country"],
            item["club_id"],
            item.get("registration_date") or None,
            item.get("lang") or None,
            item["original_member_id"],
            item.get("birthday") or None # Check if "birthday" exists
        )
        users[item["member_id"]] = user.to_dict()  # Convert User object to dictionary

    return users

In [39]:
def get_profile_link(userid: int) -> str:
    from importlib import reload
    import secretslocal
    reload(secretslocal)

    url = f'https://enphysionhealthllc.virtuagym.com/member-management/member/{userid}//member'

    headers = {
        'Accept': 'application/json, text/plain, */*',
        'Accept-Language': 'en-US,en;q=0.9,ru;q=0.8',
        'Connection': 'keep-alive',
        'Cookie': f'{secretslocal.COOKIE}',
        'Referer': 'https://enphysionhealthllc.virtuagym.com/web-app/member/43950940?is_redirect=1',
        'Sec-Fetch-Dest': 'empty',
        'Sec-Fetch-Mode': 'cors',
        'Sec-Fetch-Site': 'same-origin',
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36',
        'sec-ch-ua': '"Chromium";v="118", "Google Chrome";v="118", "Not=A?Brand";v="99"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"macOS"'
    }

    response = requests.get(url, headers=headers)
    # print(response.content)
    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        parsed_data = response.json()
        if parsed_data is None:
            # print('WARNING', 'User', userid, 'experienced a null parsed data response.')
            return None
        else:
            try:
                profile_link = parsed_data['data']['profile']['profile_link']
                return profile_link
            except:
                # print('WARNING', 'User', userid, 'experienced an error in when extracting profile link from response.')
                return None
    else:
    #    print(f'{response.status_code} {response.text}')
       return None

In [40]:
def get_account_manager(userid):
    from importlib import reload
    import requests
    import secretslocal
    reload(secretslocal)

    url = f'https://enphysionhealthllc.virtuagym.com/member-management/member/{userid}//account-manager'

    headers = {
        'Accept': 'application/json, text/plain, */*',
        'Accept-Language': 'en-US,en;q=0.9,ru;q=0.8',
        'Connection': 'keep-alive',
        'Cookie': f'{secretslocal.COOKIE}',
        'Referer': 'https://enphysionhealthllc.virtuagym.com/web-app/member/43950940?is_redirect=1',
        'Sec-Fetch-Dest': 'empty',
        'Sec-Fetch-Mode': 'cors',
        'Sec-Fetch-Site': 'same-origin',
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36',
        'sec-ch-ua': '"Chromium";v="118", "Google Chrome";v="118", "Not=A?Brand";v="99"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"macOS"'
    }

    response = requests.get(url, headers=headers)
    # print(response.content)
    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        parsed_data = response.json()
        # print(parsed_data)
        if parsed_data is None:
            # print('WARNING', 'User', userid, 'experienced a null parsed data response.')
            return None
        else:
            try:
                acct_mgr = parsed_data['data'][0]['name']
                return acct_mgr     
            except:
                # print('WARNING', 'User', userid, 'experienced an error in when extracting profile link from response.')
                return None
    else:
       print(f'{response.status_code} {response.text}')
       return None
    

In [55]:
def update_dict_with_profile_link_and_acct_mgr(users: dict) -> dict:
    # userid --> {name: blah, birthday: balh, asdf: asdf}
    for userid, userdict in users.items():
        if userdict['firstname'] == 'Joshua' and userdict['lastname'] == 'Kaplan':
            continue
        
        if not userdict['is_pro']:
            continue 
        
        profile_link = get_profile_link(userid)

        acct_mgr = get_account_manager(userid) or 'Joshua Kaplan'

        userdict['profile_link'] = profile_link
        userdict['acct_mgr'] = acct_mgr

        userdict = {k: userdict[k] for k in userdict.keys() if userdict[k] is not None}

        users[userid] = userdict
    
    return users

In [42]:
def split_users_into_active_and_inactive(users: dict) -> tuple[dict, dict]:
    users_with_profile_link = dict()
    users_without_profile_link = dict()

    for userid, userdict in users.items():
        if 'profile_link' in userdict:
            users_with_profile_link[userid] = userdict
        else:
            users_without_profile_link[userid] = userdict
    
    return users_with_profile_link, users_without_profile_link

In [43]:
def create_output_dataframe(data: dict) -> dict:
    users = list(data.keys())
    firstnames = [v['firstname'] for _, v in data.items()]
    lastnames = [v['lastname'] for _, v in data.items()]
    days_since_last_activity = [v['days_since_last_active'] for _, v in data.items()]
    acct_mgrs = [v['acct_mgr'] if 'acct_mgr' in data[k] else None for k, v in data.items()]

    df = pd.DataFrame({
        'member_id': users,
        'firstname': firstnames,
        'lastname': lastnames,
        'days_since_last_active': days_since_last_activity,
        'account_manager': acct_mgrs
    })
    dfs = dict()
    # Group by account_manager and iterate over groups
    for account_manager, group_df in df.groupby('account_manager'):
        # Save each group to a separate CSV file
        timestamp = datetime.datetime.now().strftime('%Y%m%d%H%M%S')
        filename = f'./final_outputs/{account_manager}_{timestamp}.csv'
        dfs[account_manager] = group_df
        group_df.to_csv(filename, index=False)
        print(f'Exported file to {filename}.')
    return dfs


In [63]:
def get_recent_activity_data(user_profile_link: str) -> requests.Response:
    from importlib import reload
    import secretslocal
    reload(secretslocal)

    # Define the request headers
    headers = {
        'Accept': '*/*',
        'Accept-Encoding': 'gzip, deflate, br',
        'Accept-Language': 'en-US,en;q=0.9,ru;q=0.8',
        'Connection': 'keep-alive',
        'Cookie': f'{secretslocal.COOKIE}',
        'Host': 'enphysionhealthllc.virtuagym.com',
        'Referer': 'https://enphysionhealthllc.virtuagym.com/user/cfcsandeep-1b4bd803/exercise',
        'Sec-Fetch-Dest': 'empty',
        'Sec-Fetch-Mode': 'cors',
        'Sec-Fetch-Site': 'same-origin',
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36',
        'X-Requested-With': 'XMLHttpRequest',
        'sec-ch-ua': '"Not/A)Brand";v="99", "Google Chrome";v="115", "Chromium";v="115"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"macOS"'
    }

    # Define the URL
    url = f'https://enphysionhealthllc.virtuagym.com{user_profile_link}/exercise/ajax?action=get_overview_data&period=1&offset=0'

    # Make the GET request
    response = requests.get(url, headers=headers)
    return response


In [59]:
def compute_days_since_last_activity(script: str) -> Optional[int]:
    # Use regex to find the categories and data values
    categories_match = re.search(r"categories: \[(.*?)\]", script)
    data_match = re.search(r"data: \[(.*?)\]", script)

    # Extract the matched strings and convert them into lists
    if categories_match and data_match:
        # categories = json.loads(f"[{categories_match.group(1)}]")
        data = json.loads(f"[{data_match.group(1)}]")

        # Find the most recent non-zero integer from right to left
        recent_non_zero_index_from_end = None
        for index, value in enumerate(reversed(data)):
            if value > 0:
                recent_non_zero_index_from_end = index
                break
        
        print(f"Most recent non-zero integer is {value} at index {recent_non_zero_index_from_end} from the end.")
        return recent_non_zero_index_from_end
    else:
        print("Failed to extract categories or data.")
    return -1

In [60]:
def get_last_thirty_days_activity(users):
    for userid, userdict in users.items():
        proflink = userdict['profile_link']
        ret = get_recent_activity_data(proflink)
        day_cnt = compute_days_since_last_activity(str(ret.content))
        userdict['days_since_last_active'] = day_cnt
        users[userid] = userdict
    return users        

In [61]:
def run():
    start = datetime.datetime.now()
    
    print('Getting all members.')
    users = get_all_club_members()
    step1 = datetime.datetime.now()
    print(f'Completed step in: {step1-start}')

    print('Getting profile links for members.')
    users = update_dict_with_profile_link_and_acct_mgr(users)
    step2 = datetime.datetime.now()
    print(f'Completed step in: {step2-step1}')

    print('Removing users without profile links.')
    users_with_profile_links, _ = split_users_into_active_and_inactive(users)
    step3 = datetime.datetime.now()
    print(f'Completed step in: {step3-step2}')

    print('Getting last 30 days activity per user and computing days since last active.')
    users_with_activity_calendar = get_last_thirty_days_activity(users_with_profile_links)
    step4 = datetime.datetime.now()
    print(f'Completed step in: {step4-step3}')

    print('Creating and outputting dataframe.')
    output_dfs = create_output_dataframe(users_with_activity_calendar)
    step5 = datetime.datetime.now()
    print(f'Completed step in: {step5-step4}')
    
    end = datetime.datetime.now()
    print(f'Finished process in: {end-start}')

In [62]:
run()

Getting all members.
Completed step in: 0:00:00.942224
Getting profile links for members.
401 {"message":"Unauthorized"}
401 {"message":"Unauthorized"}
401 {"message":"Unauthorized"}
401 {"message":"Unauthorized"}
Completed step in: 0:03:13.618814
Removing users without profile links.
Completed step in: 0:00:00.000555
Getting last 30 days activity per user and computing days since last active.
b'<script type="text/javascript">\n\t$(\'#next_btn\').hide();\n\t$(\'#prev_btn\').hide();\n</script>\n<input type="hidden" id="csrf_token" value=""/>\n<div id="graph_container">\n\t<div style="height:10px;">\n\t\t<a id="prev_btn" href="javascript:load_graph(-1, 1);" style="float:left;">&lt;&nbsp;previous</a>\n\t\t<div id="month_year" style="float:left;font-size:22px;line-height:22px;text-align:center; width:710px; height:24px;">May - Jun</div>\n\t\t<a id="next_btn" href="javascript:load_graph(1, 1);" style="float:right; display:none;">next&nbsp;&gt;</a>\n\t</div>\n\t<div id="charts">\n\t\t<div id

In [53]:
# Testing
users = get_all_club_members()
users[41468290]
get_profile_link(41468290)
tmp = {41468290: users[41468290]}
tmp = update_dict_with_profile_link_and_acct_mgr(tmp)
users_with_profile_links, users_without_profile_links = split_users_into_active_and_inactive(tmp)
users_with_activity = get_last_thirty_days_activity(users_with_profile_links)
users_with_activity[41468290]

/user/nnizel-b4263e14
b'<script type="text/javascript">\n\t$(\'#next_btn\').hide();\n\t$(\'#prev_btn\').hide();\n</script>\n<input type="hidden" id="csrf_token" value=""/>\n<div id="graph_container">\n\t<div style="height:10px;">\n\t\t<a id="prev_btn" href="javascript:load_graph(-1, 1);" style="float:left;">&lt;&nbsp;previous</a>\n\t\t<div id="month_year" style="float:left;font-size:22px;line-height:22px;text-align:center; width:710px; height:24px;">May - Jun</div>\n\t\t<a id="next_btn" href="javascript:load_graph(1, 1);" style="float:right; display:none;">next&nbsp;&gt;</a>\n\t</div>\n\t<div id="charts">\n\t\t<div id="left_chart"></div>\n\t\t\n<div id="graph" style="width:825px; height:230px;"></div>\n\n<script type="text/javascript">\n\t$(function () {\n\t\t$(\'#graph\').highcharts({\n\t\t\tchart: {\n\t\t\t\ttype: \'column\'\n\t\t\t},\n\t\t\ttitle: {\n\t\t\t\ttext: \'\'\n\t\t\t},\n\t\t\tcolors: [\n    \t       \t\'#000000\', \n    \t       \t\'#c1e905\',\n\t\t\t\t\'#47a72a\',\n\t\t\t

{'member_id': 41468290,
 'firstname': 'Nancy',
 'lastname': 'Nizel',
 'active': True,
 'is_pro': True,
 'gender': 'u',
 'email': 'nnizel@gmail.com',
 'member_since': 1683676800000,
 'timestamp_edit': 1683726281242,
 'country': 'US',
 'club_id': 56930,
 'registration_date': 1683725238,
 'lang': 'en',
 'original_member_id': 0,
 'profile_link': '/user/nnizel-b4263e14',
 'acct_mgr': 'Joshua Kaplan',
 'days_since_last_active': 1}